In [5]:
from twelvedata import TDClient
import pandas as pd
from datetime import datetime
import requests
import time
import logging

In [6]:
# Set up logging
logging.basicConfig(
    level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s"
)

# Initialize Twelve Data client
TD_API_KEY = "fe9301beb6ab4bacaf066516f7f80164"
td = TDClient(apikey=TD_API_KEY)

In [7]:
# Load S&P 500 symbols from Wikipedia
def fetch_sp500_symbols():
    url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
    df_list = pd.read_html(url)
    sp500_df = df_list[0]
    return set(sp500_df["Symbol"].str.upper())


In [8]:
# Get fundamentals from Twelve Data
def get_fundamentals_td(symbol):
    try:
        fundamentals = td.get_fundamentals(symbol=symbol).as_json()
        return fundamentals.get("metrics", {}), fundamentals.get("name", "N/A")
    except Exception as e:
        logging.warning(f"Failed to fetch data for {symbol}: {e}")
        return None, None

In [9]:
get_fundamentals_td('AAPL')

2025-05-05 13:27:22,066 - WARNING - Failed to fetch data for AAPL: 'TDClient' object has no attribute 'get_fundamentals'


(None, None)

In [10]:
BASE_URL = "https://api.twelvedata.com"

def get_fundamentals(symbol):
    try:
        url = f"{BASE_URL}/fundamentals"
        params = {"symbol": symbol, "apikey": TD_API_KEY}
        response = requests.get(url, params=params)
        data = response.json()
        return data.get("metrics", {}), data.get("name", "N/A")
    except Exception as e:
        logging.warning(f"Failed to fetch data for {symbol}: {e}")
        return None, None

In [11]:
get_fundamentals('AAPL')

2025-05-05 13:27:22,305 - WARNING - Failed to fetch data for AAPL: Extra data: line 1 column 5 (char 4)


(None, None)

In [12]:
def get_fundamentals_td(symbol):
    try:
        fundamentals = td.get_fundamentals(symbol=symbol).as_json()
        return fundamentals.get("metrics", {})
    except Exception:
        return None

In [13]:
get_fundamentals("AAPL")

2025-05-05 13:27:22,548 - WARNING - Failed to fetch data for AAPL: Extra data: line 1 column 5 (char 4)


(None, None)

In [14]:
data = td.get_statistics(symbol="AAPL").as_json()

In [15]:
# P/B
PB = data["statistics"]["valuations_metrics"]["price_to_book_mrq"]

In [16]:
data

{'meta': {'symbol': 'AAPL',
  'name': 'Apple Inc.',
  'currency': 'USD',
  'exchange': 'NASDAQ',
  'mic_code': 'XNGS',
  'exchange_timezone': 'America/New_York'},
 'statistics': {'valuations_metrics': {'market_capitalization': 3067066712064,
   'enterprise_value': 3116759777280,
   'trailing_pe': 31.936237,
   'forward_pe': 24.711191,
   'peg_ratio': 24.711191,
   'price_to_sales_ttm': 7.660657,
   'price_to_book_mrq': 45.92932,
   'enterprise_to_revenue': 7.785,
   'enterprise_to_ebitda': 22.444},
  'financials': {'fiscal_year_ends': '2024-09-28',
   'most_recent_quarter': '2025-03-31',
   'gross_margin': 0.47050619238876246,
   'profit_margin': 0.24301,
   'operating_margin': 0.31028998,
   'return_on_assets_ttm': 0.23809999,
   'return_on_equity_ttm': 1.38015,
   'income_statement': {'revenue_ttm': 400366010368,
    'revenue_per_share_ttm': 26.455,
    'quarterly_revenue_growth': 0.051,
    'gross_profit_ttm': 186699005952,
    'ebitda': 138866000000,
    'net_income_to_common_ttm':

In [17]:
income_statement = td.get_income_statement(symbol="AAPL").as_json()

In [18]:
# Annual Sales (Revenue)
sales = income_statement["income_statement"][0]["sales"]
# Net Income
net_income = income_statement["income_statement"][0]["net_income"]

In [36]:
sales

391035000000

In [37]:
net_income

93736000000

In [19]:
balance_sheet = td.get_balance_sheet(symbol="AAPL").as_json()

In [20]:
# Total Assets
total_assets = balance_sheet["balance_sheet"][0]["assets"]["total_assets"]

In [31]:
total_assets

364980000000

In [21]:
# Total Liabilities
total_liabilities = balance_sheet["balance_sheet"][0]["liabilities"]["total_liabilities"]

In [30]:
total_liabilities

308030000000

In [32]:
total_assets - total_liabilities

56950000000

In [34]:
data.get("statistics", {}).get("stock_statistics", {}).get("shares_outstanding")

15116786000

In [35]:
56950000000 / 15116786000

3.7673351994266504

In [22]:
# Total Current Assets
total_current_assets = balance_sheet["balance_sheet"][0]["assets"]["current_assets"]["total_current_assets"]

In [23]:
# Total Current Liabilities
total_current_liabilities = balance_sheet["balance_sheet"][0]["liabilities"]["current_liabilities"]["total_current_liabilities"]

In [24]:
# Long-term Debt
long_term_debt = balance_sheet["balance_sheet"][0]["liabilities"]["non_current_liabilities"]["long_term_debt"]

In [25]:
total_current_assets

152987000000

In [26]:
total_current_liabilities

176392000000

In [27]:
# Net Current Assets
net_current_assests = total_current_assets - total_current_liabilities

In [28]:
import requests

response = requests.get(
    "https://api.twelvedata.com/ad?apikey=fe9301beb6ab4bacaf066516f7f80164&interval=1year&symbol=AAPL&format=JSON"
)

In [29]:
response.text

'{"code":400,"message":"Invalid **interval** provided: 1year. Supported intervals: 1min, 5min, 15min, 30min, 45min, 1h, 2h, 4h, 8h, 1day, 1week, 1month","status":"error","meta":{"symbol":"AAPL","interval":"1year","exchange":""}}'